# Week5 Crawling Assignment

* 네이버 증권 [국내증시 시가총액 페이지](https://finance.naver.com/sise/sise_market_sum.nhn)에 있는 기업 리스트를 읽어옵니다. 
    이 리스트에서 크게 1) 기업명, 2) 링크를 읽어오면 됩니다.
* 위의 2) 링크를 통해 기업의 상세 컨텐츠에 접근할 수 있으며, 이 상세 컨텐츠의 투자정보를 읽어오면 됩니다.

In [1]:
import requests
from bs4 import BeautifulSoup

## 1. 기업 리스트 수집

**실습 1. get_finance_list() 작성**
- Case1-2까지의 코드

In [2]:
def get_finance_list(html):
    
    soup = BeautifulSoup(html)
    
    elements = soup.select('.box_type_l tbody tr')
    
    finance_list = []
    for element in elements:
        link_element = element.select_one('a')
        result = dict()
        if link_element:
            result['title'] = link_element.text
            result['url'] = 'https://finance.naver.com' + link_element['href']
            finance_list.append(result)
        
    return finance_list

### Unit Tests

**Case 1-1** - 시가총액 페이지에 아무런 기업이 없을 경우

In [3]:
html = """
<div class="box_type_l">
    <table>
        <tbody>
        </tbody>
    </table>
</div>
"""

finance_list = get_finance_list(html)

assert len(finance_list) == 0

**Case 1-2** - 시가총액 페이지에 기업이 한 개 있을 경우

In [4]:
html = """
<div class="box_type_l">
    <table>
        <tbody>
            <tr>
            </tr>
            <tr>
                <td>
                    <a href="/item/main.nhn?code=000001" class="tltle"></a>
                </td>
            </tr>
        </tbody>
    </table>
</div>
"""

finance_list = get_finance_list(html)

assert len(finance_list) == 1

**Case 1-3** - 시가총액 페이지에 기업이 두 개 있을 경우

In [5]:
html = """
<div class="box_type_l">
    <table>
        <tbody>
            <tr>
            </tr>
            <tr>
                <td>
                    <a href="/item/main.nhn?code=000001" class="tltle"></a>
                </td>
            </tr>
            <tr>
                <td>
                    <a href="/item/main.nhn?code=000002" class="tltle"></a>
                </td>
            </tr>
        </tbody>
    </table>
</div>
"""

finance_list = get_finance_list(html)

assert len(finance_list) == 2

**Case 1-4** - 한 개의 기업이 있는 시가총액 페이지 에서 기업 이름을 가져오기

In [6]:
html = """
<div class="box_type_l">
    <table>
        <tbody>
            <tr>
            </tr>
            <tr>
                <td>
                    <a href="/item/main.nhn?code=000001" class="title">첫 번째 기업</a>
                </td>
            </tr>
        </tbody>
    </table>
</div>
"""

finance_list = get_finance_list(html)

assert len(finance_list) == 1
first_finance = finance_list[0]
assert first_finance['title'] == "첫 번째 기업"

**Case 1-5** - 한 개의 기업이 있는 시가총액 페이지 에서 기업 이름과 링크를 가져오기

In [7]:
html = """
<div class="box_type_l">
    <table>
        <tbody>
            <tr>
            </tr>
            <tr>
                <td>
                    <a href="/item/main1.nhn" class="tltle">첫 번째 기업</a>
                </td>
            </tr>
        </tbody>
    </table>
</div>
"""

finance_list = get_finance_list(html)
 
assert len(finance_list) == 1

first_finance = finance_list[0]

assert first_finance['title'] == "첫 번째 기업"
assert first_finance['url'] == "https://finance.naver.com/item/main1.nhn"

**Case 1-6** - 두 개의 기업이 있는 시가총액 페이지 에서 기업 이름과 링크를 가져오기

In [8]:
html = """
<div class="box_type_l">
    <table>
        <tbody>
            <tr>
            </tr>
            <tr>
                <td>
                    <a href="/item/main1.nhn" class="tltle">첫 번째 기업</a>
                </td>
            </tr>
            <tr>
                <td>
                    <a href="/item/main2.nhn" class="tltle">두 번째 기업</a>
                </td>
            </tr>
        </tbody>
    </table>
</div>
"""

finance_list = get_finance_list(html)

assert len(finance_list) == 2

first_finance = finance_list[0]

assert first_finance['title'] == "첫 번째 기업"
assert first_finance['url'] == "https://finance.naver.com/item/main1.nhn"

first_finance = finance_list[1]

assert first_finance['title'] == "두 번째 기업"
assert first_finance['url'] == "https://finance.naver.com/item/main2.nhn"

**Case 1-7** - 실제 네이버 증권 시가총액 한 페이지를 크롤링하기

아래 코드를 실행시켜 다음과 같이 결과가 잘 출력되는지 확인해주세요.  
결과가 잘 출력되지 않는다면 get_finance_list()를 수정해야 합니다!
![%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-02-24%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2012.12.31.png](attachment:%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-02-24%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2012.12.31.png)

In [9]:
# finance_list_url에 네이버 증권 url을 저장합니다.
finance_list_url = "https://finance.naver.com/sise/sise_market_sum.nhn?&page=1"

# 위 URL로 접속한다음 그 응답 결과를 가져옵니다. 이를 response라는 이름의 변수에 할당합니다.
response = requests.get(finance_list_url)

# 국내증시 시가총액 페이지에서 기업 리스트를 읽어옵니다.
finance_list = get_finance_list(response.content)

# 이 기업 리스트를 전부 출력합니다.
finance_list

[{'title': '삼성전자',
  'url': 'https://finance.naver.com/item/main.nhn?code=005930'},
 {'title': 'SK하이닉스',
  'url': 'https://finance.naver.com/item/main.nhn?code=000660'},
 {'title': 'NAVER',
  'url': 'https://finance.naver.com/item/main.nhn?code=035420'},
 {'title': 'LG화학',
  'url': 'https://finance.naver.com/item/main.nhn?code=051910'},
 {'title': '삼성전자우',
  'url': 'https://finance.naver.com/item/main.nhn?code=005935'},
 {'title': '현대차',
  'url': 'https://finance.naver.com/item/main.nhn?code=005380'},
 {'title': '삼성바이오로직스',
  'url': 'https://finance.naver.com/item/main.nhn?code=207940'},
 {'title': '삼성SDI',
  'url': 'https://finance.naver.com/item/main.nhn?code=006400'},
 {'title': '카카오',
  'url': 'https://finance.naver.com/item/main.nhn?code=035720'},
 {'title': '셀트리온',
  'url': 'https://finance.naver.com/item/main.nhn?code=068270'},
 {'title': '기아차',
  'url': 'https://finance.naver.com/item/main.nhn?code=000270'},
 {'title': '현대모비스',
  'url': 'https://finance.naver.com/item/main.nhn?

**실습2. get_all_finance_list() 작성**

**Case 1-8** - 실제 네이버 증권 시가총액 전체 페이지를 크롤링하기

약 1500가량의 모드 회사의 기업 이름과, 링크가 잘 가져와지도록 get_all_finance_list()를 구현해보세요.  
(시간이 조금 걸릴 수 있습니다)

In [10]:
def get_all_finance_list():
    all_finance_list = []
    # 초기값 설정
    count = 1
    finance_list = [1]
    # 크롤링 할 페이지기 없다면 반복을 중단
    while finance_list:
        
        finance_list_url = "https://finance.naver.com/sise/sise_market_sum.nhn?&page="+ str(count)
        response = requests.get(finance_list_url)
        finance_list = get_finance_list(response.content)
        
        all_finance_list += finance_list
        count +=1
        
    return all_finance_list
    
all_finanace_list = get_all_finance_list()
all_finanace_list

[{'title': '삼성전자',
  'url': 'https://finance.naver.com/item/main.nhn?code=005930'},
 {'title': 'SK하이닉스',
  'url': 'https://finance.naver.com/item/main.nhn?code=000660'},
 {'title': 'NAVER',
  'url': 'https://finance.naver.com/item/main.nhn?code=035420'},
 {'title': 'LG화학',
  'url': 'https://finance.naver.com/item/main.nhn?code=051910'},
 {'title': '삼성전자우',
  'url': 'https://finance.naver.com/item/main.nhn?code=005935'},
 {'title': '현대차',
  'url': 'https://finance.naver.com/item/main.nhn?code=005380'},
 {'title': '삼성바이오로직스',
  'url': 'https://finance.naver.com/item/main.nhn?code=207940'},
 {'title': '삼성SDI',
  'url': 'https://finance.naver.com/item/main.nhn?code=006400'},
 {'title': '카카오',
  'url': 'https://finance.naver.com/item/main.nhn?code=035720'},
 {'title': '셀트리온',
  'url': 'https://finance.naver.com/item/main.nhn?code=068270'},
 {'title': '기아차',
  'url': 'https://finance.naver.com/item/main.nhn?code=000270'},
 {'title': '현대모비스',
  'url': 'https://finance.naver.com/item/main.nhn?

In [11]:
len(all_finanace_list)

1578

## 2. 기업 상세 정보 수집

## - 주가 정보

- 주가
- 시가
- 시가총액
- 시가총액순위

**실습 3. get_finance_detail() 작성**

In [12]:
import re

def get_finance_detail(html):
    
    soup = BeautifulSoup(html)
    finance = dict()
    value_li = soup.select_one('.rate_info')
    value = value_li.select_one('.no_today .blind')
    open_value = value_li.select_one('.no_info .blind')
    
    market_cap_li = soup.select_one('.tab_con1')
    market_cap = market_cap_li.select_one('#_market_sum')
    market_cap_rank = market_cap_li.select_one('.link_site')
    
    if value:
        finance['value'] = int(re.sub('[^0-9]', "", value.text))
    else:
        finance['value'] = value
    
    if open_value:
        finance['open_value'] = int(re.sub('[^0-9]', "", open_value.text))
    else:
        finance['open_value'] = open_value

    if market_cap:
        finance['market_cap(억원)'] = int(re.sub('[^0-9]', "", market_cap.text))
    else:
        finance['market_cap(억원)'] = market_cap        
        
    if market_cap_rank:
        finance['market_cap_rank'] = re.sub('[^0-9가-힣]', "", market_cap_rank.parent.parent.select_one('td').text)
    else:
        finance['market_cap_rank'] = market_cap_rank
        
    return finance

### Unit Tests


**Case 2-1** - 기업정보 페이지에 아무런 내용이 없을 경우

In [13]:
html = """
<div class="rate_info">
</div>
<div class="tab_con1">
</div>
"""

finance = get_finance_detail(html)

assert finance['value'] == None

**Case 2-2** - 기업정보 페이지에서 주가 가져오기

In [14]:
html = """
<div class="rate_info">
    <div class="no_today">
        <div class="blind">
            82,000
        </div>
    </div>
</div>
<div class="tab_con1">
</div>
"""

finance = get_finance_detail(html)

assert finance['value'] == 82000

**Case 2-3** - 기업정보 페이지에서 시가를 추가로 가져오기

`<div class="rate_info">`에서 "시가"라는 단어를 찾고 html 태그간의 부모-자식 관계를 이용하여 찾아보세요.

In [15]:
html = """
<div class="rate_info">
    <div class="no_today">
        <div class="blind">
            82,000
        </div>
    </div>
    <table class="no_info">
        <tr>
            <td class="first">
                <span class="sptxt sp_txt3">시가</span>
                            <em class="no_up">

                    <span class="blind">81,200</span>
                    <span class="no8">8</span>
                    <span class="no1">1</span>
                    <span class="shim">,</span>
                    <span class="no2">2</span>
                    <span class="no0">0</span>
                    <span class="no0">0</span>

                </em>
            </td>
        </tr>
    </table>
</div>
<div class="tab_con1">
</div>
"""

finance = get_finance_detail(html)

assert finance['value'] == 82000

assert finance['open_value'] == 81200

**Case 2-4** - 기업정보 페이지에서 시가총액을 추가로 가져오기

`<div id="tab_con1" class="tab_con1">`에서 "시가총액"이라는 단어를 찾고 html 태그간의 부모-자식 관계를 이용하여 찾아보세요.

In [16]:
html = """
<div class="rate_info">
</div>
<div id="tab_con1" class="tab_con1">
    <div>
        <table>
            <caption>시가총액</caption>
            <tr class="strong">
                <th scope="row">시가총액</th>
                <td><em id="_market_sum">
                            320조
                                5,773
                    </em>억원
                </td>
            </tr>
        </table>
    </div>
</div>
"""

finance = get_finance_detail(html)

assert finance['market_cap(억원)'] == 3205773

**Case 2-5-1** - 기업정보 페이지에서 시가총액순위를 추가로 가져오기

`<div id="tab_con1" class="tab_con1">`에서 "시가총액순위"라는 단어를 찾고 html 태그간의 부모-자식 관계를 이용하여 찾아보세요.

In [17]:
html = """
<div class="rate_info">
</div>
<div id="tab_con1" class="tab_con1">
    <div>
        <table>
            <tr>
                <th scope="row">
                    <a class="link_site" href="/sise/sise_market_sum.nhn">시가총액순위</a>
                </th>
                <td>
                    코스피 <em>1</em>위
                </td>
            </tr>
        </table>
    </div>
</div>
"""

finance = get_finance_detail(html)

assert finance['market_cap_rank'] == '코스피1위'

**Case 2-5-2** - 기업정보 페이지에 시가총액순위가 없는 경우

In [18]:
html = """
<div class="rate_info">
</div>
<div id="tab_con1" class="tab_con1">
    <div>
        <table>
            <tr>
            </tr>
        </table>
    </div>
</div>
"""

finance = get_finance_detail(html)

assert finance['market_cap_rank'] == None

**Case 2-6** - 실제 네이버 증권 상세 페이지를 크롤링하기

아래 코드를 실행시켜 다음과 같이 결과가 잘 출력되는지 확인해주세요.  
결과가 잘 출력되지 않는다면 get_finance_detail()를 수정해야 합니다!
![%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-02-24%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2012.21.27.png](attachment:%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-02-24%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%2012.21.27.png)

In [19]:
def get_all_finance_detail(response):
    
    finance_detail = get_finance_detail(response.content)
    
    finance = {
        'value' : finance_detail['value'],
        'open_value' : finance_detail['open_value'],
        'market_cap(억원)': finance_detail['market_cap(억원)'],
        'market_cap_rank': finance_detail['market_cap_rank'],
    }

    return finance

In [20]:
finance_url = "https://finance.naver.com/item/main.nhn?code=005930"

response = requests.get(finance_url)

finance = get_all_finance_detail(response)

finance

{'market_cap(억원)': 5092225,
 'market_cap_rank': '코스피1위',
 'open_value': 82000,
 'value': 85300}

### 3. 전체 페이지 읽어오기

**실습 4. 모든 기업리스트에 대한 상세 정보 crawling**  
finance_summary_list 안에 있는 모든 기업 정보를 가져와 finanace_list에 담도록 구현합니다.

In [21]:
# get_all_finance_list함수를 활용하여 전체페이지에서 기업 요약정보(기업이름, url)를 받아옵니다.
finance_summary_list = get_all_finance_list()
    
finance_list = []
#######################################################
# finance_summary_list 안에 있는 모든 기업 정보를 가져와 finanace_list에 담도록 구현합니다.
for finance_summary in finance_summary_list:
    
    finance_url = finance_summary['url']
    response = requests.get(finance_url)
    finance = get_all_finance_detail(response)
    
    finance = {
        'title': finance_summary['title'],
        'url'  : finance_summary['url'],
        'value': finance['value'],
        'open_value': finance['open_value'],
        'market_cap(억원)': finance['market_cap(억원)'],
        'market_cap_rank': finance['market_cap_rank']     
    }
    finance_list.append(finance)

#######################################################

finance_list

[{'market_cap(억원)': 5092225,
  'market_cap_rank': '코스피1위',
  'open_value': 82000,
  'title': '삼성전자',
  'url': 'https://finance.naver.com/item/main.nhn?code=005930',
  'value': 85300},
 {'market_cap(억원)': 1081084,
  'market_cap_rank': '코스피2위',
  'open_value': 136000,
  'title': 'SK하이닉스',
  'url': 'https://finance.naver.com/item/main.nhn?code=000660',
  'value': 148500},
 {'market_cap(억원)': 629129,
  'market_cap_rank': '코스피3위',
  'open_value': 374000,
  'title': 'NAVER',
  'url': 'https://finance.naver.com/item/main.nhn?code=035420',
  'value': 383000},
 {'market_cap(억원)': 628272,
  'market_cap_rank': '코스피4위',
  'open_value': 860000,
  'title': 'LG화학',
  'url': 'https://finance.naver.com/item/main.nhn?code=051910',
  'value': 890000},
 {'market_cap(억원)': 609759,
  'market_cap_rank': '코스피5위',
  'open_value': 72700,
  'title': '삼성전자우',
  'url': 'https://finance.naver.com/item/main.nhn?code=005935',
  'value': 74100},
 {'market_cap(억원)': 523487,
  'market_cap_rank': '코스피6위',
  'open_value':

### 4. CSV로 저장하기

In [22]:
import pandas as pd

finance_list_df = pd.DataFrame(finance_list)

finance_list_df

,title,url,value,open_value,market_cap(억원),market_cap_rank
0,삼성전자,https://finance.naver.com/item/main.nhn?code=0...,85300,82000,5092225,코스피1위
1,SK하이닉스,https://finance.naver.com/item/main.nhn?code=0...,148500,136000,1081084,코스피2위
2,NAVER,https://finance.naver.com/item/main.nhn?code=0...,383000,374000,629129,코스피3위
3,LG화학,https://finance.naver.com/item/main.nhn?code=0...,890000,860000,628272,코스피4위
4,삼성전자우,https://finance.naver.com/item/main.nhn?code=0...,74100,72700,609759,코스피5위
...,...,...,...,...,...,...
1573,동양3우B,https://finance.naver.com/item/main.nhn?code=0...,24950,25500,22,코스피1574위
1574,KBSTAR 200건설,https://finance.naver.com/item/main.nhn?code=2...,11350,10980,20,None
1575,KBSTAR 팔라듐선물인버스(H),https://finance.naver.com/item/main.nhn?code=3...,5070,5245,20,None
1576,KBSTAR 미국장기국채선물인버스2X(합성 H),https://finance.naver.com/item/main.nhn?code=2...,6600,6465,20,None


In [23]:
finance_list_df.to_csv("finance.csv")